# WNTR Tutorial

## Imports
Import WNTR and additional Python packages that are needed for the tutorial
- Numpy is required to support numerical analysis
- Matplotlib is used for graphics
- Scipy is used to define lognormal fragility curves

In [ ]:
import numpy as np
import matplotlib.pylab as plt
from scipy.stats import lognorm
import wntr

In [ ]:
# The following function is used below to extract public properties and methods on WNTR objects
def dir_public(obj):
    return [name for name in dir(obj) if not name.startswith('_')]

# 1. Water Network Model

The `WaterNetworkModel` object defines the water distribution system and simulation options. The object can be created from an EPANET INP file.

In [ ]:
# Create water network model from an INP file
wn = wntr.network.WaterNetworkModel('../networks/Net3.inp')

In [ ]:
# Basic description of the model
wn.describe(level=1)

In [ ]:
# Explore properties and methods associated with the WaterNetworkModel
dir_public(wn)

In [ ]:
# Basic network graphic
ax = wntr.graphics.plot_network(wn)

## Nodes
Node describe network junctions, tanks, and reservoirs

In [ ]:
# Print the names of all junctions, tanks, and reservoirs
print("Node names", wn.node_name_list)

In [ ]:
# Print the names of just tanks
print("Tank names", wn.tank_name_list)

In [ ]:
# Get a tank object, check its type, print properties and methods
tank = wn.get_node('1')
print(type(tank))
print(dir_public(tank))

In [ ]:
# Change the max level
print("Original max level", tank.max_level)
tank.max_level = 10
print("New max level", tank.max_level)

In [ ]:
# Add a junction

# Remove a junction


## Links

Links describe pipes, pumps, and valves

In [ ]:
# Print the names of all links
print("Link names", wn.link_name_list)

In [ ]:
# Print the names of just head pumps
print("Head pump names", wn.head_pump_name_list)

In [ ]:
# Get the name of links connected to a specific node
connected_links = wn.get_links_for_node('229')
print('Links connected to node 229 =', connected_links)

In [ ]:
# Get a pipe object, check its type, print properties and methods
pipe = wn.get_link('105')
print(type(pipe))
print(dir_public(pipe))

In [ ]:
# Change the diameter
print("Original diameter", pipe.diameter)
pipe.diameter = 10
print("New diameter", pipe.diameter)

In [ ]:
# Add a pipe

# Remove a pipe

## Demands and Patterns
Junctions can have multiple demands which are stored in a `demand_timeseries_list`. The following illustrates how to
* Compute expected demand (which accounts for base demand, demand patterns, and demand multiplier)
* Compute average expected demand (average value for a 24 hour period, also accounts for base demand, demand patterns, and demand multiplier)
* Add demands to a junction
* Modify demand base value and pattern
* Remove demands from a junction
* Plot expected and simulated demands

In [ ]:
# Compute expected demand
expected_demand = wntr.metrics.expected_demand(wn)
print(expected_demand.head())

In [ ]:
# Compute and plot average expected demand and 
AED = wntr.metrics.average_expected_demand(wn)
print(AED.head())
ax = wntr.graphics.plot_network(wn, node_attribute=AED, node_range=(0,0.025))

In [ ]:
# Identify junctions with zero demand
zero_demand = AED[AED == 0].index
print(zero_demand)
ax = wntr.graphics.plot_network(wn, node_attribute=list(zero_demand))

In [ ]:
# Get the demands on Junction 15
junction = wn.get_node('15')
print(junction.demand_timeseries_list)

In [ ]:
# Get the pattern associated with the demand
pattern = wn.get_pattern(junction.demand_timeseries_list[0].pattern_name)
pattern

In [ ]:
# Add a demand
junction.add_demand(base=0.015, pattern_name='1')

# Modify the base value of the original demand
junction.demand_timeseries_list[0].base_value = 0.005

# Add a pattern
wn.add_pattern('new', [1,1,1,0,0,0,1,0,0.5,0.5,0.5,1])
junction.demand_timeseries_list[0].pattern_name = "new"
print(junction.demand_timeseries_list)

In [ ]:
# Plot original and modified expected demands
new_expected_demand = wntr.metrics.expected_demand(wn) 

plt.figure()
ax = expected_demand.loc[0:48*3600, "15"].plot()
new_expected_demand.loc[0:48*3600, "15"].plot(ax=ax)

## Curves

Curves include pump head curves, tank volume curves, and pump efficiency curves.  The following example illustrates how to work with pump head curves and tank volume curves.

In [ ]:
# Get a head pump object
pump = wn.get_link('10')
print(type(pump))

# Plot the head pump curve
ax = wntr.graphics.plot_pump_curve(pump)

In [ ]:
# Get the head curve and print the points
pump_curve_name = pump.pump_curve_name
curve = wn.get_curve(pump_curve_name)
print(curve.points)

# Modify the curve points and replot the pump curve
curve.points = [(0.10, 20)]
print(curve.points)
ax = wntr.graphics.plot_pump_curve(pump)

In [ ]:
wn.add_curve('new_tank_curve', 'VOLUME', [
   (1,  0),
   (2,  60),
   (3,  188),
   (4,  372),
   (5,  596),
   (6,  848),
   (7,  1114),
   (8,  1379),
   (9,  1631),
   (10, 1856),
   (11, 2039),
   (12, 2168),
   (13, 2228)])
tank = wn.get_node('2')
tank.vol_curve_name = 'new_tank_curve'
ax = wntr.graphics.plot_tank_volume_curve(tank)

## Controls

In [ ]:
for name, controls in wn.controls():
    print(name, controls)

In [ ]:
# Modify a control
control = wn.get_control('control 18')


In [ ]:
# Add a control

# Remove a control

In [ ]:
# Convert controls to rules
wn.convert_controls_to_rules()
for name, controls in wn.controls():
    print(name, controls)

## Model I/O

In [ ]:
# Write a INP file from the WaterNetworkModel
wntr.network.io.write_inpfile(wn, 'modified_network.inp', units='LPS')

In [ ]:
wn_dict = wn.to_dict()
#print(wn_dict)

In [ ]:
#to_graph
#write json

## Queries

In [ ]:
# Return all pipe diameters
all_pipe_diameters = wn.query_link_attribute('diameter')
print(all_pipe_diameters.head())

In [ ]:
# Return pipes diameters > 12 inches
large_pipe_diameters = wn.query_link_attribute('diameter', np.greater, 12*0.0254)
print(large_pipe_diameters.head())

In [ ]:
# Plot large pipes
ax = wntr.graphics.plot_network(wn, link_attribute=large_pipe_diameters, node_size=0, 
                           link_width=2, title="Pipes with diameter > 12 inches")

## Loops and generators
Loops and generators are commonly used to modify network components or run stochastic simulations

In [ ]:
# Loop over tank names and objects with a generator
for name, tank in wn.tanks():
    print("Max level for tank", name, "=", tank.max_level)

In [ ]:
# Loop over tank names and then get the associated tank object
for name in wn.tank_name_list:
    tank = wn.get_node(name)
    print("Max level for tank", name, "=", tank.max_level)

## Coordinates

## Pipe breaks and leaks

In [ ]:
wn = wntr.morph.split_pipe(wn, '123', '123_B', '123_node')
leak_node = wn.get_node('123_node')
leak_node.add_leak(wn, area=0.05, start_time=2*3600, end_time=12*3600)

# 2. Hydraulic and Water Quality Simulations

WNTR includes 2 simulators: the `EpanetSimulator` and the `WNTRSimulator`.  Both include the ability to run pressure dependent demand (PDD) or demand-driven (DD) hydraulic simulation.  Only the EpanetSimulator runs water quality simulations.

In [ ]:
# Create water network model from an INP file
wn = wntr.network.WaterNetworkModel('../networks/Net3.inp')

## Simulation options

In [ ]:
wn.options

In [ ]:
# Set the simulation duration to 4 days
wn.options.time.duration = 4*24*3600

In [ ]:
# PDD


## EPANET and WNTR Simulators

In [ ]:
# Simulate hydraulics using EPANET
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

## Simulation results

In [ ]:
# Plot timeseries of tank levels
tank_levels = results.node['pressure'].loc[:,wn.tank_name_list]
tank_levels.plot(title='Tank level')

# Plot timeseries of pump flowrates
pump_flowrates = results.link['flowrate'].loc[:,wn.pump_name_list]
pump_flowrates.plot(title='Pump flowrate')

# Plot pressure at hour 5 on the network
pressure_at_5hr = results.node['pressure'].loc[5*3600, :]
ax = wntr.graphics.plot_network(wn, node_attribute=pressure_at_5hr, node_size=30, title='Pressure at 5 hours')

## Reset initial conditions

In [ ]:
wn.reset_initial_values()

# 3. Resilience Metrics

## Topographic

## Hydraulic

# 4. Fragility Curves

In [ ]:
FC = wntr.scenario.FragilityCurve()
FC.add_state('Minor', 1, {'Default': lognorm(0.5,scale=0.3)})
FC.add_state('Major', 2, {'Default': lognorm(0.5,scale=0.7)})
ax = wntr.graphics.plot_fragility_curve(FC, xlabel='Peak Ground Acceleration (g)')

# 5. Network Skeletonization

In [ ]:
# Create water network model from an INP file
wn = wntr.network.WaterNetworkModel('../networks/Net3.inp')
wn.describe(level=1)

In [ ]:
# Skeletonize the network using a 12 inch pipe diameter threshold
skel_wn = wntr.morph.skeletonize(wn, 12*0.0254)
skel_wn.describe(level=1)

In [ ]:
# Plot the original and skeletonized networks
ax = wntr.graphics.plot_network(wn, node_size=10, title='Original')
ax = wntr.graphics.plot_network(skel_wn, node_size=10, title='Skeletonized')

In [ ]:
# Simulate hydraulics on the original and skeletonized models and plot average pressure
sim = wntr.sim.EpanetSimulator(wn)
results_original = sim.run_sim()

sim = wntr.sim.EpanetSimulator(skel_wn)
results_skel = sim.run_sim()

ax = results_original.node['pressure'].mean(axis=1).plot(label='Original')
ax = results_skel.node['pressure'].mean(axis=1).plot(ax=ax, label='Skeletonized')
plt.legend()

# 6. Valve Segmentation

In [ ]:
# Create water network model from an INP file
wn = wntr.network.WaterNetworkModel('../networks/Net3.inp')

In [ ]:
# Create a N-2 strategic valve layer
valve_layer = wntr.network.generate_valve_layer(wn, 'strategic', 2)
ax = wntr.graphics.plot_valve_layer(wn, valve_layer, add_colorbar=False)

In [ ]:
# Identify nodes and links that are in each valve segment
G = wn.to_graph()
node_segments, link_segments, seg_sizes = wntr.metrics.topographic.valve_segments(G, valve_layer)
print('Node segments')
print(node_segments)
print()
print('Link segments')
print(link_segments)
print()
print('Segment sizes')
print(seg_sizes)

In [ ]:
N = seg_sizes.shape[0]
cmap = wntr.graphics.random_colormap(N) # random color map helps visualize segments
ax = wntr.graphics.plot_network(wn, link_attribute=link_segments, node_size=0, link_width=2, link_range=[0,N], 
                                link_cmap=cmap, link_colorbar_label='Segment ID')
ax = wntr.graphics.plot_valve_layer(wn, valve_layer, add_colorbar=False, ax=ax)

# 7. Geospatial Capabilities

In [ ]:
# Create water network model from an INP file
wn = wntr.network.WaterNetworkModel('../networks/Net3.inp')

In [ ]:
wn_gis = wntr.network.to_gis(wn)
print(wn_gis.pipes.head())